In [ ]:
!pip install torch==2.4.0 transformers==4.46.2 --break-system-packages

In [ ]:
!python -m pip install --upgrade setuptools --break-system-packages

`pillow==1.25.0` on their [HF page](https://huggingface.co/AIDC-AI/Ovis2-1B) doesnt work with my python; hoping `1.26.4` will work

In [ ]:
!pip install numpy==1.26.4 pillow==10.3.0 --break-system-packages

In [ ]:
!pip install flash-attn==2.7.0.post2 --no-build-isolation --break-system-packages

In [ ]:
!pip uninstall -y torchvision --break-system-packages
!pip install torchvision --break-system-packages

In [ ]:
!pip install --upgrade torchvision safetensors --break-system-package

In [ ]:
!git lfs install
!git clone https://huggingface.co/AIDC-AI/Ovis2-1B

In [1]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM

In [2]:
# load model
model = AutoModelForCausalLM.from_pretrained("Ovis2-1B",
                                             torch_dtype=torch.bfloat16,
                                             multimodal_max_length=32768,
                                             trust_remote_code=True).cuda()
text_tokenizer = model.get_text_tokenizer()
visual_tokenizer = model.get_visual_tokenizer()

In [3]:
# single-image input
image_path = 'example.jpg'
images = [Image.open(image_path)]
max_partition = 9
text = '''
      Give the json data format of the slip text inside the image; 
      give the name of the store the logo shows, the name of the branch separate, 
      a list of items purchased that has the cost to the right of the row, 
      and the final total the slip shows.'''
query = f'<image>\n{text}'

In [4]:
# format conversation
prompt, input_ids, pixel_values = model.preprocess_inputs(query, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, text_tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

In [5]:
# generate output
with torch.inference_mode():
    gen_kwargs = dict(
        max_new_tokens=1024,
        do_sample=False,
        top_p=None,
        top_k=None,
        temperature=None,
        repetition_penalty=None,
        eos_token_id=model.generation_config.eos_token_id,
        pad_token_id=text_tokenizer.pad_token_id,
        use_cache=True
    )
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    output = text_tokenizer.decode(output_ids, skip_special_tokens=True)
    print(f'{output}')

{"storeName": "Pick n Pay", "branchName": "Mini Market Oakdene", "itemsPurchased": ["SOUP PACK", "ALMONDS SLICED 100GR", "5% Pensioners Discount5.0%"], "total": 54.13}
